In [1]:
import numpy as np
from joblib import Parallel, delayed
from math import sqrt
from itertools import product

In [32]:
def power_it(arr, idx, n=2):
    return arr[idx] ** n

In [40]:
def parfor(arr, func, n_jobs=2, *args, **kwargs):
    """ 
    Parallel for loop for numpy arrays
    """
    idx = list(product(*(range(s) for s in arr.shape)))
    results = Parallel(n_jobs=n_jobs)(delayed(func)(arr, i, *args, **kwargs) for i in idx)
    return np.array(results).reshape(arr.shape + (-1, )).squeeze()

In [41]:
my_array = np.arange(100).reshape(10, 10)

In [42]:
aa = parfor(my_array, power_it)

In [46]:
aa[8, 8]

9801

In [49]:
i, j = np.random.randint(0, 9, 2)
assert aa[i, j] == power_it(my_array, (i, j)) 

In [50]:
%timeit parfor(my_array, power_it, n=3, n_jobs=1)

100 loops, best of 3: 1.82 ms per loop


In [51]:
%timeit parfor(my_array, power_it, n=3, n_jobs=2)

10 loops, best of 3: 129 ms per loop


In [52]:
%timeit parfor(my_array, power_it, n=3, n_jobs=4)

10 loops, best of 3: 134 ms per loop
